In [1]:
from jira import JIRA
from dotenv import load_dotenv
import getpass
import os

load_dotenv()

JIRA_URL = os.environ.get("JIRA_URL")
EMAIL = os.environ.get("EMAIL")
API_TOKEN = os.environ.get("JIRA_API_TOKEN")

jira = JIRA(
    server=JIRA_URL,
    basic_auth=(EMAIL, API_TOKEN)
)

# Test connection
user = jira.current_user()
print(f"Connected as: {user}")

Connected as: 712020:4da4c4ca-c19f-47de-bbb6-3c4574de0abd


# Accès aux tickets

In [2]:
issues = jira.search_issues(
    'project = SMP ORDER BY created DESC',
    maxResults=5
)

for issue in issues:
    print(issue.key, issue.fields.summary)

SMP-8 Test issue from Python
SMP-7 Se connecter avec les bons identifiants
SMP-1 Connexion


In [6]:
for issue in issues:
    print(jira.issue(issue.key, fields="key,summary,content")) # ?

SMP-8
SMP-7
SMP-1


In [ ]:
issue = jira.issue("SMP-7")

print(issue.key)
print(issue.fields.summary)
print(issue.fields.status.name)